In [219]:
class Node:
    def __init__(self, char, frequency, left=None, right=None):
        self.char = char
        self.frequency = frequency
        self.left = left
        self.right = right
        self.code = ''
        
#   this function recursively traverses the whole tree to assign code value for each node, it assigns '0' for the left child and '1' for the right child
    def encode_node(self, code):
        self.code = code
        if self.left is not None:
            self.left.encode_node(code + '0')
        if self.right is not None:
            self.right.encode_node(code + '1')


In [220]:
class Heap:
    def __init__(self):
        self.cbt = []  # initialize arrays
        self.next_index = 0  # denotes next index where new element should go

    def insert(self, node):
        # insert element at the next index
        self.cbt.append(node)

        # heapify
        self._up_heapify()

        # increase index by 1
        self.next_index += 1


    def remove(self):
        
        if self.size() == 0:
            return None
        
        to_remove = self.cbt.pop(0)
        self.next_index -= 1

        return to_remove

    def size(self):
        return self.next_index 

    def _up_heapify(self): # up_heapify rearranges the heap according to the frequency value of the node 
        
        child_index = self.next_index

        while child_index >= 1:
            parent_index = (child_index - 1) // 2
            parent_element = self.cbt[parent_index]
            child_element = self.cbt[child_index]

            if parent_element.frequency > child_element.frequency:
                self.cbt[parent_index] = child_element
                self.cbt[child_index] = parent_element

                child_index = parent_index
            else:
                break




In [221]:

#   step 1:  build the huffman tree using a min heap
def huffman_tree(data):
    freq_table = {}
    # first get each character with its frequency
    if len(data) < 1:
        print("you have to provide at least one letter")
        return
    for char in data:
        if char in freq_table:
            freq_table[char] += 1
        else:
            freq_table[char] = 1
            
    # second build the heap 
    dict_to_arr = list(freq_table.items())
    sorted_arr = sorted(dict_to_arr, key=lambda x: x[1])
    priority_q = Heap()
    for item in sorted_arr:
        node = Node(item[0], item[1])
        priority_q.insert(node)
    
#     build the huffman tree from the heap by removing 2 items with the lowest frequency and create a new internal node haaving a combined frequency of the 2 removed items
    while len(priority_q.cbt) > 1:
        node1 = priority_q.remove()
        node2 = priority_q.remove()
        if node1 is None or node2 is None:
            break
        combined_freq = node1.frequency + node2.frequency
        combined_node = Node(None, combined_freq, left=node1, right=node2) # combined node does't have a character 
        priority_q.insert(combined_node)
        
    return priority_q.cbt 



In [222]:
#   step2: encode the data
   # to encode the data we need to call the huffman tree on the data which will return the periority queue having one element which is the root for the whole tree

def huffman_encoding(data):
    if len(data) < 1:
        return
    codes = {}
    encoded_data = ""
    tree = huffman_tree(data)
    tree_root = tree[0]
    tree_root.encode_node("")  # start assigning codes from the root which itself will not be assigned any binary code
   
    #this function traverses the tree to build the codes dictionary (the key is the chareacter and the value is the code)
    create_code_table(tree_root, codes)
    
    # loop over each character in the data and get its code from the codes dictionary and concatenate it into one string
    for char in data:
        encoded_data += codes[char]
    
    return encoded_data

# a helper function to build the codes dictionary
def create_code_table(node, codes):
    if node.char and node.code:
        codes[node.char] = node.code
    if node.left is not None:
        create_code_table(node.left, codes)
    if node.right is not None:
        create_code_table(node.right, codes)



In [223]:
def huffman_decoding(encoded_data, tree):
    if tree is None:
        return
    decoded_string = ""
    current_node = tree[0]

    for bit in encoded_data:
        if bit == '0':
            current_node = current_node.left
        else:
            current_node = current_node.right
        
        if current_node.char != None: # only leaf nodes have characters so when we reach it we concatenate the char to the decoded string then reset the node to the root node for next iteration
            decoded_string += current_node.char
            current_node = tree[0]  # Reset to the root for the next character, we'll start again from the root to the leaf node having the value of the character 
    return decoded_string



In [224]:
import sys
## Test Case 1
data = "The bird is the word"
print ("The size of the data is: {}\n".format(sys.getsizeof(data)))
print ("The content of the data is: {}\n".format(data))

encoded_data = huffman_encoding(data)
print ("The size of the encoded data is: {}\n".format(sys.getsizeof(int(encoded_data, base=2))))
print ("The content of the encoded data is: {}\n".format(encoded_data))

tree = huffman_tree(data)
decoded_data = huffman_decoding(encoded_data, tree)
print ("The size of the decoded data is: {}\n".format(sys.getsizeof(decoded_data)))
print ("The content of the decoded data is: {}\n".format(decoded_data))


The size of the data is: 69

The content of the data is: The bird is the word

The size of the encoded data is: 36

The content of the encoded data is: 1000111011110111001000001010011000101001110111110111101111001101001010

The size of the decoded data is: 69

The content of the decoded data is: The bird is the word



In [225]:
## Test Case 2
data = ""
print ("The size of the data is: {}\n".format(sys.getsizeof(data)))
print ("The content of the data is: {}\n".format(data))

encoded_data = huffman_encoding(data)
print ("The size of the encoded data is: {}\n".format(sys.getsizeof(encoded_data)))
print ("The content of the encoded data is: {}\n".format(encoded_data))

tree = huffman_tree(data)
decoded_data = huffman_decoding(encoded_data, tree)
print ("The size of the decoded data is: {}\n".format(sys.getsizeof(decoded_data)))
print ("The content of the decoded data is: {}\n".format(decoded_data))

The size of the data is: 49

The content of the data is: 

The size of the encoded data is: 16

The content of the encoded data is: None

you have to provide at least one letter
The size of the decoded data is: 16

The content of the decoded data is: None



In [226]:
## Test Case 3
data = "aaaaabbbbbbbbbccccccccAAAAAAAABBBBBBBCCCCCCDDDDDDdddddddEEEEEEEeeeeeeeHHHHHHHHHhRRRRRRRRLLkiihbbhdbvhibidhgbvhfbgibfhgb"
print ("The size of the data is: {}\n".format(sys.getsizeof(data)))
print ("The content of the data is: {}\n".format(data))

encoded_data = huffman_encoding(data)
print ("The size of the encoded data is: {}\n".format(sys.getsizeof(int(encoded_data, base=2))))
print ("The content of the encoded data is: {}\n".format(encoded_data))

tree = huffman_tree(data)
decoded_data = huffman_decoding(encoded_data, tree)
print ("The size of the decoded data is: {}\n".format(sys.getsizeof(decoded_data)))
print ("The content of the decoded data is: {}\n".format(decoded_data))


The size of the data is: 168

The content of the data is: aaaaabbbbbbbbbccccccccAAAAAAAABBBBBBBCCCCCCDDDDDDdddddddEEEEEEEeeeeeeeHHHHHHHHHhRRRRRRRRLLkiihbbhdbvhibidhgbvhfbgibfhgb

The size of the encoded data is: 92

The content of the encoded data is: 010001000100010001001100110011001100110011001100110011001111111111111111111111111111111110001000100010001000100010001000011101110111011101110111011101010101010101010101010100100010001000100010001010101010101010101010101010101101110111011101110111011101111011101110111011101110111010111011101110111011101110111011101101101001100110011001100110011001100100111100111100111000000000011011001100011010101100000100110000011000000101001100011011000001001100001111000011000001100000110110001101100

The size of the decoded data is: 168

The content of the decoded data is: aaaaabbbbbbbbbccccccccAAAAAAAABBBBBBBCCCCCCDDDDDDdddddddEEEEEEEeeeeeeeHHHHHHHHHhRRRRRRRRLLkiihbbhdbvhibidhgbvhfbgibfhgb

